# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import seaborn as sns
import re, string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data loading

In [2]:
data_path = '/content/drive/MyDrive/ZIwG P/citations.csv'
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 120)

df = pd.read_csv(data_path)
df = df.dropna()
df = df.reset_index(drop=True)
# df.info()


df['label'].replace({'Fałsz': 0, 'Prawda': 1, 'Manipulacja': 2, 'Nieweryfikowalne': 3}, inplace=True)
df = df[(df['label'] == 0) | (df['label'] == 1)]
# X.head()
# y.head(10)
df

,content,author,label
0,O ile my podnieśliśmy pensję minimalną w czasie 8 lat? (…) o ponad 90 proc.,Izabela Leszczyna,0
1,W rankingu praworządności World Justice Project w 2020 r. Polska znów spadła na niższą pozycję.,Hanna Gill-Piątek,1
2,"Na Uniwersytecie Warszawskim powstał taki raport, który jest oczywiście państwu znany, z którego wyraźnie wynika, że...",Dariusz Rosati,1
3,Średnia emerytura w Polsce kobiet i mężczyzn różni się aż o 1000 zł.,Małgorzata Kidawa-Błońska,1
4,"Proszę mi pokazać (…) jedną osobę skazaną przez Trybunał Stanu od czasu, kiedy Trybunał Stanu w Polsce, wolnej, tak ...",Paweł Kukiz,0
...,...,...,...
4330,"Chroniliście SKOK-i, chroniliście swoich kolesi. Dlatego z budżetu państwa ponad 4,5 mld zł trzeba było im wypłacić.",Borys Budka,1
4332,"Jeżeli chodzi o penalizację homoseksualizmu, to w Polsce nigdy czegoś takiego nie było. Ani za I Rzeczypospolitej, a...",Kosma Złotowski,0
4333,Rządowy Fundusz Inwestycji Lokalnych na Pomorzu – największy beneficjent gmina miasta Gdańsk.,Marcin Horała,1
4334,"Zarówno WHO, EMA, czyli Europejska Agencja Leków, jak i polskie instytucje, w tym Urząd Rejestracji Produktów Leczni...",Michał Dworczyk,1


# Text preprocessing

In [3]:
def remove_punctuation(text):
    no_punctuation_text = "".join([char for char in text if char not in string.punctuation])
    return no_punctuation_text


def tokenize(text):
    tokens = re.split('\W+', text)
    text = [word for word in tokens]
    return text


def lemmatizing(text):
    #TODO when we get polish lemmatizer
    pass


#TODO when we decide what how to preprocess text
# now it is just copy-paste from online course
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens]
    return text

In [4]:
df['clean_text'] = df['content'].apply(lambda x: clean_text(x))
df.head()

,content,author,label,clean_text
0,O ile my podnieśliśmy pensję minimalną w czasie 8 lat? (…) o ponad 90 proc.,Izabela Leszczyna,0,"[o, ile, my, podnieśliśmy, pensję, minimalną, w, czasie, 8, lat, o, ponad, 90, proc]"
1,W rankingu praworządności World Justice Project w 2020 r. Polska znów spadła na niższą pozycję.,Hanna Gill-Piątek,1,"[w, rankingu, praworządności, world, justice, project, w, 2020, r, polska, znów, spadła, na, niższą, pozycję]"
2,"Na Uniwersytecie Warszawskim powstał taki raport, który jest oczywiście państwu znany, z którego wyraźnie wynika, że...",Dariusz Rosati,1,"[na, uniwersytecie, warszawskim, powstał, taki, raport, który, jest, oczywiście, państwu, znany, z, którego, wyraźni..."
3,Średnia emerytura w Polsce kobiet i mężczyzn różni się aż o 1000 zł.,Małgorzata Kidawa-Błońska,1,"[średnia, emerytura, w, polsce, kobiet, i, mężczyzn, różni, się, aż, o, 1000, zł]"
4,"Proszę mi pokazać (…) jedną osobę skazaną przez Trybunał Stanu od czasu, kiedy Trybunał Stanu w Polsce, wolnej, tak ...",Paweł Kukiz,0,"[proszę, mi, pokazać, jedną, osobę, skazaną, przez, trybunał, stanu, od, czasu, kiedy, trybunał, stanu, w, polsce, w..."


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], test_size=0.25)

In [6]:
X_train

695                                        [w, prywatnych, przychodniach, w, ramach, abonamentów, leczy, się, 15, mln, polaków]
2399    [korweta, gawron, miała, być, przerobiona, przez, rząd, po, i, psl, na, żyletki, odrodziła, się, jako, orp, ślązak, ...
2834                                              [99, przypadków, aborcji, w, polsce, to, są, tak, zwane, aborcje, eugeniczne]
209                                                                                    [już, 2, mln, polaków, jest, bez, pracy]
2839    [anna, zalewska, ja, wiem, ale, do, ginekologa, to, nie, jest, taki, problemmonika, olejnik, najpierw, trzeba, iść, ...
                                                                 ...                                                           
2438    [są, badania, które, wskazują, że, ze, względu, na, demografię, za, 20, lat, będzie, 25, mln, mniej, polaków, w, wie...
2794                                       [problem, z, sędziami, tk, polega, na, tym, że, są, to, osoby

## Vectorization

### N-gram

In [7]:
# range = (2, 2)

# ngram_vect = CountVectorizer(ngram_range=range)
# X_ngram = ngram_vect.fit_transform(X['content'])

# X_ngram_df = pd.DataFrame(X_ngram.toarray())
# X_ngram_df.columns = ngram_vect.get_feature_names()
# X_ngram_df

### TF-IDF

In [8]:
# tfidf_vect = TfidfVectorizer()
# tfidf_vect.fit(X_train['content'])

# X_train_vect = tfidf_vect.transform(X_train['content'])
# X_test_vect = tfidf_vect.transform(X_test['content'])

In [9]:
# X_test_vect[0].toarray()

# Model nr 1

# Model nr 2

# Deep Learning

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_seq_pad = pad_sequences(X_train_seq, len(max(X_train_seq, key=len)))
X_test_seq_pad = pad_sequences(X_test_seq, len(max(X_train_seq, key=len)))



## RNN model

In [31]:
batch_size = 64

model = Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, 64))
model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.3, return_sequences=True, recurrent_initializer='glorot_uniform'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 64)          824256    
_________________________________________________________________
lstm_8 (LSTM)                (None, None, 32)          12416     
_________________________________________________________________
dense_10 (Dense)             (None, None, 64)          2112      
_________________________________________________________________
dense_11 (Dense)             (None, None, 1)           65        
Total params: 838,849
Trainable params: 838,849
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [33]:
early_stopping = EarlyStopping(monitor='loss', patience=3)

history = model.fit(X_train_seq_pad, y_train, validation_data=(X_test_seq_pad, y_test), batch_size=batch_size, epochs=32, callbacks=[early_stopping])

Epoch 1/32
42/42 [==============================] - 14s 260ms/step - loss: 0.6592 - accuracy: 0.6458 - val_loss: 0.5915 - val_accuracy: 0.7324
Epoch 2/32
42/42 [==============================] - 10s 249ms/step - loss: 0.5989 - accuracy: 0.7183 - val_loss: 0.5838 - val_accuracy: 0.7324
Epoch 3/32
42/42 [==============================] - 10s 250ms/step - loss: 0.6022 - accuracy: 0.7118 - val_loss: 0.5820 - val_accuracy: 0.7324
Epoch 4/32
42/42 [==============================] - 10s 250ms/step - loss: 0.5878 - accuracy: 0.7253 - val_loss: 0.5833 - val_accuracy: 0.7324
Epoch 5/32
42/42 [==============================] - 11s 252ms/step - loss: 0.5914 - accuracy: 0.7197 - val_loss: 0.5855 - val_accuracy: 0.7324
Epoch 6/32
42/42 [==============================] - 10s 250ms/step - loss: 0.5909 - accuracy: 0.7175 - val_loss: 0.5836 - val_accuracy: 0.7323
Epoch 7/32
42/42 [==============================] - 11s 254ms/step - loss: 0.5902 - accuracy: 0.7144 - val_loss: 0.5832 - val_accuracy: 0.7318

In [35]:
# prediction = 
# prediction = np.argmax(model.predict(X_test_seq_pad[48]))
# prediction

# plt.figure(figsize=(10,6))
# sns.heatmap(confusion_matrix(test_image_gen.classes,predictions),annot=True)

161

# Evaluation